# Tutorial 2: TransportPCE SDN Controller Setup

## Learning Objectives
- Understand Software Defined Networking (SDN) concepts for optical networks
- Learn about TransportPCE controller architecture
- Set up TransportPCE in isolated student environment
- Verify controller readiness and API accessibility
- Understand RESTCONF protocol basics

## Prerequisites
- Completed Tutorial 1 (Docker Basics and LightyNode Setup)
- Running LightyNode containers from previous tutorial
- Understanding of basic network management concepts

## Overview
TransportPCE is an OpenDaylight-based SDN controller designed specifically for optical transport networks. It provides centralized management of optical devices, path computation, and service provisioning for OpenROADM networks. TransportPCE acts as a centralized controller that can manage multiple devices and provide higher-level network services.

## Step 1: Student Configuration

**IMPORTANT: Use the same STUDENT_ID as in previous tutorials!**

In [ ]:
# === STUDENT CONFIGURATION ===
# MUST MATCH YOUR STUDENT_ID FROM PREVIOUS TUTORIALS!
STUDENT_ID = "1"  # Change this to your assigned student ID

# Generated configuration
NETWORK_NAME = f"student{STUDENT_ID}-network"
TPCE_CONTAINER_NAME = f"student{STUDENT_ID}-tpce"

print(f"Student ID: {STUDENT_ID}")
print(f"Network: {NETWORK_NAME}")
print(f"TransportPCE Container: {TPCE_CONTAINER_NAME}")

## Step 2: Import Required Libraries

We'll use Docker SDK for container management and requests for HTTP operations.

In [ ]:
import docker
import requests
import time
import json
from requests.auth import HTTPBasicAuth
from docker.errors import NotFound, APIError
from common_utils import StatusIndicators, print_status, print_section, DockerHelper

# Initialize Docker client and disable SSL warnings
docker_helper = DockerHelper()
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

print_status("Libraries imported successfully", StatusIndicators.SUCCESS)

## Step 3: Helper Functions for TransportPCE Management

These functions help manage TransportPCE container and operations.

In [ ]:
# Helper functions

def get_container_name(base_name):
    """Generate student-specific container name.
    Essential for multi-student environments to prevent container name conflicts."""
    return f"student{STUDENT_ID}-{base_name}"

def start_transportpce(image="linqixiao/transport-pce:karaf-master", timeout=300):
    """Start TransportPCE SDN controller container.
    This is the core function for deploying the optical network controller."""
    print(f"Starting TransportPCE container: {TPCE_CONTAINER_NAME}")
    
    try:
        # Check if container already exists
        container = docker_helper.client.containers.get(TPCE_CONTAINER_NAME)
        if container.status == "running":
            print_status(f"TransportPCE container already running", StatusIndicators.INFO)
            return container
        else:
            print_status(f"Starting existing TransportPCE container", StatusIndicators.INFO)
            container.start()
            return container
    except NotFound:
        # Create new container
        print_status(f"Creating new TransportPCE container", StatusIndicators.WORKING)
        
        container = docker_helper.client.containers.run(
            image=image,
            name=TPCE_CONTAINER_NAME,
            network=NETWORK_NAME,
            environment={
                "FEATURES": "odl-transportpce"
            },
            # Standard TransportPCE ports:
            # Port 8181: RESTCONF API
            # Port 8101: Karaf console  
            detach=True,
            remove=False
        )
        
        print_status(f"Created TransportPCE container: {TPCE_CONTAINER_NAME}", StatusIndicators.SUCCESS)
        return container

def wait_for_transportpce(container, timeout=300):
    """Wait for TransportPCE to be fully operational.
    Critical function - TransportPCE takes 2-5 minutes to start all services."""
    print_status(f"Waiting for TransportPCE to start (timeout: {timeout}s)", StatusIndicators.WORKING)
    
    # Messages that indicate TransportPCE is ready
    ready_messages = [
        b"Transportpce controller started",
        b"lighty.io and RESTCONF started"
    ]
    
    start_time = time.time()
    elapsed = 0
    
    while elapsed < timeout:
        try:
            logs = container.logs()
            
            # Check for ready messages
            for ready_msg in ready_messages:
                if ready_msg in logs:
                    print_status("TransportPCE is ready", StatusIndicators.SUCCESS)
                    return True
            
            # Show progress
            elapsed = int(time.time() - start_time)
            if elapsed % 30 == 0:  # Update every 30 seconds
                print_status(f"Still waiting... ({elapsed}s elapsed)", StatusIndicators.WORKING)
            
            time.sleep(10)
            
        except Exception as e:
            print_status(f"Error checking logs: {e}", StatusIndicators.WARNING)
            time.sleep(5)
    
    print_status(f"TransportPCE did not start within {timeout} seconds", StatusIndicators.ERROR)
    return False

print_status("Helper functions defined", StatusIndicators.SUCCESS)

## Step 4: Verify Network and Existing Containers

Make sure our student network and LightyNode containers are available.

In [ ]:
def check_student_environment():
    """Check student's network and container environment (simplified)"""
    print_section(f"Environment Check for Student {STUDENT_ID}")
    
    # Check network
    try:
        network = docker_helper.client.networks.get(NETWORK_NAME)
        print_status(f"Network {NETWORK_NAME} exists", StatusIndicators.SUCCESS)
        
        # Show network containers (simplified)
        containers = docker_helper.get_network_containers(NETWORK_NAME)
        for container in containers:
            print_status(f"  {container['name']}: {container['ip']}", StatusIndicators.INFO)
            
    except docker.errors.NotFound:
        print_status(f"Network {NETWORK_NAME} not found", StatusIndicators.ERROR)
        print_status("Please run Tutorial 1 first to create the network", StatusIndicators.INFO)
        return False

    return True

environment_ok = check_student_environment()

## Step 5: Pull TransportPCE Image

Before starting TransportPCE, we need to pull the Docker image from the repository.

In [ ]:
# Define the TransportPCE image
TPCE_IMAGE = "linqixiao/transport-pce:karaf-master"

print_status(f"Pulling TransportPCE Docker image: {TPCE_IMAGE}", StatusIndicators.WORKING)

try:
    # Pull the image
    image = docker_helper.client.images.pull(TPCE_IMAGE)
    print_status(f"Successfully pulled {TPCE_IMAGE}", StatusIndicators.SUCCESS)
    
    # Display simplified image details
    size_gb = image.attrs.get('Size', 0) / (1024*1024*1024)
    print_status(f"Image size: {size_gb:.2f} GB", StatusIndicators.INFO)
    
except Exception as e:
    print_status(f"Failed to pull {TPCE_IMAGE}", StatusIndicators.ERROR, str(e))
    print_status("Troubleshooting: Check internet connection and Docker daemon", StatusIndicators.INFO)

## Step 6: Start TransportPCE Controller

Now let's start the TransportPCE SDN controller in our isolated network.

In [ ]:
if environment_ok:
    print("Starting TransportPCE SDN Controller...\n")
    
    # Start TransportPCE
    tpce_container = start_transportpce()
    
    if tpce_container:
        print(f"\nTransportPCE container status: {tpce_container.status}")
        
        # Wait for TransportPCE to be ready
        if wait_for_transportpce(tpce_container):
            print("\n✓ TransportPCE started successfully!")
        else:
            print("\n⚠ TransportPCE may still be starting...")
            print("Last 10 lines of logs:")
            logs = tpce_container.logs().decode('utf-8', errors='ignore')
            lines = logs.split('\n')[-10:]
            for line in lines:
                if line.strip():
                    print(f"  {line}")
    else:
        print("Failed to start TransportPCE container")
        tpce_container = None
        
else:
    print("Environment check failed. Cannot start TransportPCE.")
    tpce_container = None

## Step 7: Verify TransportPCE Network Connectivity

Check that TransportPCE is properly connected to our student network.

In [ ]:
if tpce_container:
    print_status("Verifying TransportPCE network connectivity", StatusIndicators.WORKING)
    
    # Get TransportPCE IP address using docker_helper
    tpce_ip = docker_helper.get_container_ip(TPCE_CONTAINER_NAME, NETWORK_NAME)
    
    if tpce_ip:
        print_status(f"TransportPCE IP address: {tpce_ip}", StatusIndicators.SUCCESS)
        
        # Test connectivity to other containers
        try:
            # Get list of other containers in the network
            containers = docker_helper.get_network_containers(NETWORK_NAME)
            other_containers = [(c['name'], c['ip']) for c in containers 
                              if c['name'] != TPCE_CONTAINER_NAME]
            
            if other_containers:
                print_status("Testing connectivity to other containers:", StatusIndicators.INFO)
                for name, ip in other_containers[:3]:  # Test first 3
                    result = tpce_container.exec_run(f"ping -c 2 {ip}")
                    status = StatusIndicators.SUCCESS if result.exit_code == 0 else StatusIndicators.ERROR
                    print_status(f"  {name} ({ip})", status)
            else:
                print_status("No other containers to test connectivity with", StatusIndicators.INFO)
                
        except Exception as e:
            print_status(f"Error testing connectivity: {e}", StatusIndicators.ERROR)
    else:
        print_status("Could not get TransportPCE IP address", StatusIndicators.ERROR)
else:
    print_status("No TransportPCE container to test", StatusIndicators.ERROR)
    tpce_ip = None

## Step 8: Interactive Exercise - Test RESTCONF API Access

**EXERCISE**: Complete the code below to test RESTCONF API accessibility. Fill in the blanks to make the API test work.

**Learning Goals:**
- Understand HTTP methods for RESTCONF
- Learn API endpoint construction
- Practice authentication setup
- Work with HTTP status codes

In [ ]:
# EXERCISE: Complete the RESTCONF API test
# Fill in the blanks to make the API accessibility test work

# Get TransportPCE IP for the exercise
tpce_ip = docker_helper.get_container_ip(TPCE_CONTAINER_NAME, NETWORK_NAME)

if tpce_ip:
    print_status(f"Target: TransportPCE at {tpce_ip}:8181", StatusIndicators.INFO)
    
    def test_api_access_exercise():
        """Test RESTCONF API - COMPLETE THE BLANKS"""
        
        # BLANK 1: What HTTP method should we use to read data?
        # Hint: We want to GET information from the API
        http_method = "___"  # Fill in the HTTP method
        
        # BLANK 2: Complete the base URL
        # Hint: RESTCONF typically uses port 8181
        base_url = f"http://{tpce_ip}:____"  # Fill in the port number
        
        # BLANK 3: Complete the endpoint path
        # Hint: RESTCONF data access starts with /rests/data/
        endpoint = "/rests/____/network-topology:network-topology"  # Fill in the missing part
        
        # BLANK 4: Complete the authentication
        # Hint: TransportPCE default credentials are admin/admin
        username = "____"  # Fill in username
        password = "____"  # Fill in password
        
        # BLANK 5: What content type should we accept?
        # Hint: We want JSON/XML response data
        headers = {
            'Accept': 'application/____',  # Fill in content type
            'Content-Type': 'application/____'  # Fill in content type
        }
        
        # Build complete URL and show request details
        full_url = base_url + endpoint
        auth = HTTPBasicAuth(username, password)
        
        print_status("Request Details:", StatusIndicators.INFO)
        print_status(f"  Method: {http_method}", StatusIndicators.INFO)
        print_status(f"  URL: {full_url}", StatusIndicators.INFO)
        print_status(f"  Auth: {username}/{password}", StatusIndicators.INFO)
        
        try:
            # BLANK 6: Make the HTTP request
            # Hint: Use requests.get() with the parameters above
            response = requests.___(full_url, auth=auth, headers=headers, timeout=10)
            
            if response.status_code == 200:
                print_status("RESTCONF API is accessible", StatusIndicators.SUCCESS)
                print_status(f"Response size: {len(response.text)} characters", StatusIndicators.INFO)
                return True
            else:
                print_status(f"API request failed: {response.status_code}", StatusIndicators.ERROR)
                return False
                
        except Exception as e:
            print_status("API request failed", StatusIndicators.ERROR, str(e))
            return False

    print_status("Complete the blanks above, then uncomment the line below:", StatusIndicators.INFO)
    # Uncomment the line below after filling in the blanks
    # success = test_api_access_exercise()

else:
    print_status("Cannot run exercise - no TransportPCE IP address available", StatusIndicators.ERROR)

## Solution

Here's the complete solution with all blanks filled in:

In [ ]:
# EXERCISE: Complete the RESTCONF API test
# Fill in the blanks to make the API accessibility test work

# Get TransportPCE IP for the exercise
tpce_ip = docker_helper.get_container_ip(TPCE_CONTAINER_NAME, NETWORK_NAME)

if tpce_ip:
    print_status(f"Target: TransportPCE at {tpce_ip}:8181", StatusIndicators.INFO)

    def test_api_access_exercise():
        """Test RESTCONF API - COMPLETE THE BLANKS"""

        # BLANK 1: What HTTP method should we use to read data?
        # Hint: We want to GET information from the API
        http_method = "GET"  # Fill in the HTTP method

        # BLANK 2: Complete the base URL
        # Hint: RESTCONF typically uses port 8181
        base_url = f"http://{tpce_ip}:8181"  # Fill in the port number

        # BLANK 3: Complete the endpoint path
        # Hint: RESTCONF data access starts with /rests/data/
        endpoint = "/rests/data/network-topology:network-topology"  # Fill in the missing part

        # BLANK 4: Complete the authentication
        # Hint: TransportPCE default credentials are admin/admin
        username = "admin"  # Fill in username
        password = "admin"  # Fill in password

        # BLANK 5: What content type should we accept?
        # Hint: We want JSON/XML response data
        headers = {
            'Accept': 'application/xml',  # Fill in content type
            'Content-Type': 'application/xml'  # Fill in content type
        }

        # Build complete URL and show request details
        full_url = base_url + endpoint
        auth = HTTPBasicAuth(username, password)

        print_status("Request Details:", StatusIndicators.INFO)
        print_status(f"  Method: {http_method}", StatusIndicators.INFO)
        print_status(f"  URL: {full_url}", StatusIndicators.INFO)
        print_status(f"  Auth: {username}/{password}", StatusIndicators.INFO)

        try:
            # BLANK 6: Make the HTTP request
            # Hint: Use requests.get() with the parameters above
            response = requests.get(full_url, auth=auth, headers=headers, timeout=10)
            print_status("Raw Response:", StatusIndicators.INFO)
            print(response.text)

            if response.status_code == 200:
                print_status("RESTCONF API is accessible", StatusIndicators.SUCCESS)
                print_status(f"Response size: {len(response.text)} characters", StatusIndicators.INFO)
                return True
            else:
                print_status(f"API request failed: {response.status_code}", StatusIndicators.ERROR)
                return False

        except Exception as e:
            print_status("API request failed", StatusIndicators.ERROR, str(e))
            return False

    print_status("Complete the blanks above, then uncomment the line below:", StatusIndicators.INFO)
    # Uncomment the line below after filling in the blanks
    success = test_api_access_exercise()

else:
    print_status("Cannot run exercise - no TransportPCE IP address available", StatusIndicators.ERROR)

## Summary

In this tutorial, you have successfully:

### **Set Up TransportPCE SDN Controller**
- **Deployed** TransportPCE in isolated student environment
- **Verified** controller startup and readiness
- **Tested** RESTCONF API accessibility
- **Explored** initial network topology

### **Learned SDN Concepts**
- **Centralized Control**: One controller manages multiple devices
- **Network Abstraction**: High-level services vs device-level operations
- **RESTCONF Protocol**: HTTP-based network management
- **Topology Management**: Network-wide view and operations

### **Mastered Interactive API Skills**
- **HTTP Methods**: Understanding GET, POST, PUT, DELETE operations
- **URL Construction**: Building proper RESTCONF endpoints
- **Authentication**: Using Basic Auth with network controllers
- **Response Processing**: Analyzing JSON data structures
- **Postman-Style Testing**: Systematic API exploration

### **Technical Achievements**
- **Isolated Environment**: Student-specific network and containers
- **Container Orchestration**: Multi-container application setup
- **API Integration**: RESTCONF client for network operations
- **Service Discovery**: Understanding available network services

## Key Endpoints Available

Your TransportPCE controller now provides these services:
- **RESTCONF API**: `http://{tpce_ip}:8181/rests/`
- **Network Topology**: `/data/network-topology:network-topology`
- **Operations**: `/data/` (RPC endpoints)
- **OpenROADM Services**: Device mounting, service creation

## Environment Status

**Ready for Next Tutorial**: Your environment now has both LightyNode devices and TransportPCE controller running in isolation

## Next Steps

**Tutorial 3** will cover:
- Advanced RESTCONF operations
- Device mounting procedures
- Network topology manipulation
- Service provisioning basics

## Troubleshooting

Common issues and solutions:

**TransportPCE Won't Start**
- Check container logs: `docker logs {TPCE_CONTAINER_NAME}`
- Verify network connectivity
- Ensure sufficient memory/resources

**RESTCONF API Not Accessible**
- Wait longer (TransportPCE can take 2-5 minutes to fully start)
- Check container IP address
- Verify network isolation settings

**Empty Topology**
- This is expected initially
- Devices must be mounted (Tutorial 4)
- Links must be created (Tutorial 5-6)

**Exercise Completion Issues**
- Fill in ALL blanks (___) before running
- Uncomment function calls after completing blanks
- Check solutions at bottom of code cells if stuck